### Install

In [1]:
!pip install tqdm
!pip install lightgbm
!pip install xgboost
!pip install vecstack
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19877 sha256=5c05dcec597a3ba35ac46e497140ecee2515d8634c55fc09573090fb5e2b6f87
  Stored in directory: /root/.cache/pip/wheels/28/fe/0c/fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 5.0 MB/s 


### Import

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, RobustScaler, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn import linear_model, svm, ensemble
from xgboost import XGBRegressor
import lightgbm

from vecstack import StackingTransformer

from skopt import BayesSearchCV
from skopt.space import Categorical, Integer, LogN, Real

import joblib

### Prepare the data

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/220919/fifa_pay_prediction/03_work_code/Querkowe

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/220919/fifa_pay_prediction/03_work_code/Querkowe


In [4]:
train_data = pd.read_csv('../../00_original_data/FIFA_train.csv')

In [5]:
#contract_until 변수 int 형으로 변환
# 계약 연도만 추출
def func(string:object) -> int:
    """계약 연도만 추출하여 int로 반환"""
    string = string[-4:]
    return int(string)


train_data['contract_until'] = train_data['contract_until'].apply(func)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(train_data[train_data.keys().drop(['id', 'name', 'value'])], train_data['value'], random_state=42)

#### Pipeline Preprocess

In [7]:
numeric_features = ['age', 'reputation', 'stat_overall', 'stat_potential', 'stat_skill_moves']
numeric_transformer = StandardScaler() # RobustScaler

categorical_features = ['continent', 'position', 'prefer_foot']
categorical_transformer = OneHotEncoder(categories='auto', handle_unknown='ignore') 

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [8]:
preprocessor_pipe = Pipeline(steps=[('preprocessor', preprocessor)])

preprocessor_pipe.fit(x_train)

x_train_transformed = preprocessor_pipe.transform(x_train)

In [9]:
x_test_transformed = preprocessor_pipe.transform(x_test)

### Build & Learn Model

#### SVM

- Grid search : {'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'}
- Randomized search : {'C': 417.8148444007955, 'gamma': 0.3547275579413753, 'kernel': 'rbf'}
- Bayesian search : [('C', 1000.0), ('gamma', 0.0512015475365109), ('kernel', 'rbf')]

In [10]:
svr_g = svm.SVR(C=1000, gamma=0.1, kernel='rbf').fit(x_train_transformed, y_train)
svr_r = svm.SVR(C=417.8148444007955, gamma=0.3547275579413753, kernel='rbf').fit(x_train_transformed, y_train)
svr_b = svm.SVR(C=1000, gamma=0.0512015475365109, kernel='rbf').fit(x_train_transformed, y_train)

In [11]:
print('svm regressor with grid search :', mean_squared_error(y_test, svr_g.predict(x_test_transformed))**.5)
print('svm regressor with rand search :', mean_squared_error(y_test, svr_r.predict(x_test_transformed))**.5)
print('svm regressor with baye search :', mean_squared_error(y_test, svr_b.predict(x_test_transformed))**.5)

svm regressor with grid search : 6657087.995002561
svm regressor with rand search : 6799938.573693998
svm regressor with baye search : 6622588.779257085


#### Linear SVM

- Grid search : {'C': 1000, 'random_state': 42}
- Randomized search : {'C': 524.8859871129727, 'random_state': 42}
- Bayesian search : [('C', 999.8265739695903), ('random_state', 42)]

In [12]:
lsvr_g = svm.LinearSVR(C=1000, random_state=42).fit(x_train_transformed, y_train)
lsvr_r = svm.LinearSVR(C=524.8859871129727, random_state=42).fit(x_train_transformed, y_train)
lsvr_b = svm.LinearSVR(C=999.8265739695903, random_state=42).fit(x_train_transformed, y_train)

In [13]:
print('lsvm regressor with grid search :', mean_squared_error(y_test, lsvr_g.predict(x_test_transformed))**.5)
print('lsvm regressor with rand search :', mean_squared_error(y_test, lsvr_r.predict(x_test_transformed))**.5)
print('lsvm regressor with baye search :', mean_squared_error(y_test, lsvr_b.predict(x_test_transformed))**.5)

lsvm regressor with grid search : 5511990.828885629
lsvm regressor with rand search : 5822254.725812118
lsvm regressor with baye search : 5512656.161270678


#### Extra Trees

- Grid search : {'max_depth': 5, 'n_estimators': 1000, 'n_jobs': -1, 'random_state': 42} 
- Randomized search : {'max_depth': 4, 'n_estimators': 538, 'n_jobs': -1, 'random_state': 42}
- Bayesian search : [('max_depth', 8), ('n_estimators', 724), ('n_jobs', -1), ('random_state', 42)]

In [14]:
xtr_g_param = {'max_depth': 5, 'n_estimators': 1000, 'n_jobs': -1, 'random_state': 42}
xtr_r_param = {'max_depth': 4, 'n_estimators': 538, 'n_jobs': -1, 'random_state': 42}
xtr_b_param = {'max_depth': 8, 'n_estimators': 724, 'n_jobs': -1, 'random_state': 42}

In [15]:
xtr_g = ensemble.ExtraTreesRegressor(**xtr_g_param).fit(x_train_transformed, y_train)
xtr_r = ensemble.ExtraTreesRegressor(**xtr_r_param).fit(x_train_transformed, y_train)
xtr_b = ensemble.ExtraTreesRegressor(**xtr_b_param).fit(x_train_transformed, y_train)

In [16]:
print('extra trees regressor with grid search :', mean_squared_error(y_test, xtr_g.predict(x_test_transformed))**.5)
print('extra trees regressor with rand search :', mean_squared_error(y_test, xtr_r.predict(x_test_transformed))**.5)
print('extra trees regressor with baye search :', mean_squared_error(y_test, xtr_b.predict(x_test_transformed))**.5)

extra trees regressor with grid search : 1822018.0061618278
extra trees regressor with rand search : 2161332.733954486
extra trees regressor with baye search : 1296636.8223825246


#### Random Forest

- Grid search : {'max_depth': 5, 'n_estimators': 1000, 'n_jobs': -1, 'random_state': 42}
- Randomized search : {'max_depth': 4, 'n_estimators': 443, 'n_jobs': -1, 'random_state': 42}
- Bayesian search : [('max_depth', 8), ('n_estimators', 20), ('n_jobs', -1), ('random_state', 42)]

In [17]:
rfr_g_param = {'max_depth': 5, 'n_estimators': 1000, 'n_jobs': -1, 'random_state': 42}
rfr_r_param = {'max_depth': 4, 'n_estimators': 443, 'n_jobs': -1, 'random_state': 42}
rfr_b_param = {'max_depth': 8, 'n_estimators': 20, 'n_jobs': -1, 'random_state': 42}

In [18]:
rfr_g = ensemble.RandomForestRegressor(**rfr_g_param).fit(x_train_transformed, y_train)
rfr_r = ensemble.RandomForestRegressor(**rfr_r_param).fit(x_train_transformed, y_train)
rfr_b = ensemble.RandomForestRegressor(**rfr_b_param).fit(x_train_transformed, y_train)

In [19]:
print('random forest regressor with grid search :', mean_squared_error(y_test, rfr_g.predict(x_test_transformed))**.5)
print('random forest regressor with rand search :', mean_squared_error(y_test, rfr_r.predict(x_test_transformed))**.5)
print('random forest regressor with baye search :', mean_squared_error(y_test, rfr_b.predict(x_test_transformed))**.5)

random forest regressor with grid search : 1509488.2967198638
random forest regressor with rand search : 1771082.2232638933
random forest regressor with baye search : 1238439.1237164813


#### Gradient Boost

- Grid search : {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000, 'random_state': 42}
- Randomized search : {'learning_rate': 0.049577508855568365, 'max_depth': 3, 'n_estimators': 925, 'random_state': 42}
- Bayesian search : [('learning_rate', 0.06344498415616949), ('max_depth', 5), ('n_estimators', 129), ('random_state', 42)]

In [20]:
gbr_g_param = {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000, 'random_state': 42}
gbr_r_param = {'learning_rate': 0.049577508855568365, 'max_depth': 3, 'n_estimators': 925, 'random_state': 42}
gbr_b_param = {'learning_rate': 0.06344498415616949, 'max_depth': 5, 'n_estimators': 129, 'random_state': 42}

In [21]:
gbr_g = ensemble.GradientBoostingRegressor(**gbr_g_param).fit(x_train_transformed, y_train)
gbr_r = ensemble.GradientBoostingRegressor(**gbr_r_param).fit(x_train_transformed, y_train)
gbr_b = ensemble.GradientBoostingRegressor(**gbr_b_param).fit(x_train_transformed, y_train)

In [22]:
print('gradient boost regressor with grid search :', mean_squared_error(y_test, gbr_g.predict(x_test_transformed))**.5)
print('gradient boost regressor with rand search :', mean_squared_error(y_test, gbr_r.predict(x_test_transformed))**.5)
print('gradient boost regressor with baye search :', mean_squared_error(y_test, gbr_b.predict(x_test_transformed))**.5)

gradient boost regressor with grid search : 805099.9780213628
gradient boost regressor with rand search : 854630.5583619052
gradient boost regressor with baye search : 976341.6441060889


#### Extra Gradient Boost

- Grid search : {'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000, 'random_state': 42}
- Randomized search : {'gamma': 0.04586484324176051, 'learning_rate': 0.06867462147439204, 'max_depth': 4, 'n_estimators': 628, 'random_state': 42}
- Bayesian search : [('gamma', 0.001), ('learning_rate', 0.06795626895549607), ('max_depth', 3), ('n_estimators', 632), ('random_state', 42)]

In [23]:
xgbr_g_param = {'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 1000, 'random_state': 42}
xgbr_r_param = {'gamma': 0.04586484324176051, 'learning_rate': 0.06867462147439204, 'max_depth': 4, 'n_estimators': 628, 'random_state': 42}
xgbr_b_param = {'gamma': 0.001, 'learning_rate':  0.06795626895549607, 'max_depth': 3, 'n_estimators': 632, 'random_state': 42}

In [24]:
xgbr_g = XGBRegressor(**xgbr_g_param).fit(x_train_transformed, y_train)
xgbr_r = XGBRegressor(**xgbr_r_param).fit(x_train_transformed, y_train)
xgbr_b = XGBRegressor(**xgbr_b_param).fit(x_train_transformed, y_train)

[07:54:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:54:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:54:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [25]:
print('extra gradient boost regressor with grid search :', mean_squared_error(y_test, xgbr_g.predict(x_test_transformed))**.5)
print('extra gradient boost regressor with rand search :', mean_squared_error(y_test, xgbr_r.predict(x_test_transformed))**.5)
print('extra gradient boost regressor with baye search :', mean_squared_error(y_test, xgbr_b.predict(x_test_transformed))**.5)

extra gradient boost regressor with grid search : 822431.2926722522
extra gradient boost regressor with rand search : 878991.4366171896
extra gradient boost regressor with baye search : 884630.6329353064


#### Light GBM

- Grid search : {'boosting_type': 'gbdt', 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'random_state': 42}
- Randomized search : {'boosting_type': 'gbdt', 'gamma': 0.033173639397717856, 'learning_rate': 0.45702046798546164, 'max_depth': 4, 'random_state': 42}
- Bayesian search : [('boosting_type', 'gbdt'), ('gamma', 0.018521662961496488), ('learning_rate', 0.34556810356817463), ('max_depth', 6), ('random_state', 42)]

In [26]:
lgbr_g_param = {'boosting_type': 'gbdt', 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'random_state': 42}
lgbr_r_param = {'boosting_type': 'gbdt', 'gamma': 0.033173639397717856, 'learning_rate': 0.45702046798546164, 'max_depth': 4, 'random_state': 42}
lgbr_b_param = {'boosting_type': 'gbdt', 'gamma': 0.018521662961496488, 'learning_rate': 0.34556810356817463, 'max_depth': 6, 'random_state': 42}

In [27]:
lgbr_g = lightgbm.LGBMRegressor(**lgbr_g_param).fit(x_train_transformed, y_train)
lgbr_r = lightgbm.LGBMRegressor(**lgbr_r_param).fit(x_train_transformed, y_train)
lgbr_b = lightgbm.LGBMRegressor(**lgbr_b_param).fit(x_train_transformed, y_train)

In [28]:
print('light gbm regressor with grid search :', mean_squared_error(y_test, lgbr_g.predict(x_test_transformed))**.5)
print('light gbm regressor with rand search :', mean_squared_error(y_test, lgbr_r.predict(x_test_transformed))**.5)
print('light gbm regressor with baye search :', mean_squared_error(y_test, lgbr_b.predict(x_test_transformed))**.5)

light gbm regressor with grid search : 1415702.9198474837
light gbm regressor with rand search : 1212245.7860615046
light gbm regressor with baye search : 1254460.5394929934


#### SGD Regressor

- Grid search : {'alpha': 0.001, 'learning_rate': 'adaptive', 'random_state': 42}
- Randomized search : {'alpha': 0.003974513629048665, 'learning_rate': 'adaptive', 'random_state': 42}
- Bayesian search : [('alpha', 0.00021345511640311532), ('learning_rate', 'adaptive'), ('random_state', 42)]

In [29]:
sgdr_g_param = {'alpha': 0.001, 'learning_rate': 'adaptive', 'random_state': 42}
sgdr_r_param = {'alpha': 0.003974513629048665, 'learning_rate': 'adaptive', 'random_state': 42}
sgdr_b_param = {'alpha': 0.00021345511640311532, 'learning_rate': 'adaptive', 'random_state': 42}

In [30]:
sgdr_g = linear_model.SGDRegressor(**sgdr_g_param).fit(x_train_transformed, y_train)
sgdr_r = linear_model.SGDRegressor(**sgdr_r_param).fit(x_train_transformed, y_train)
sgdr_b = linear_model.SGDRegressor(**sgdr_b_param).fit(x_train_transformed, y_train)

In [31]:
print('SGD regressor with grid search :', mean_squared_error(y_test, sgdr_g.predict(x_test_transformed))**.5)
print('SGD regressor with rand search :', mean_squared_error(y_test, sgdr_r.predict(x_test_transformed))**.5)
print('SGD regressor with baye search :', mean_squared_error(y_test, sgdr_b.predict(x_test_transformed))**.5)

SGD regressor with grid search : 4011425.5227940665
SGD regressor with rand search : 4012793.499460694
SGD regressor with baye search : 4011095.7429131125


#### Linear Regressor

In [32]:
linear = linear_model.LinearRegression(n_jobs=-1).fit(x_train_transformed, y_train)

In [33]:
print('linear regression :', mean_squared_error(y_test, linear.predict(x_test_transformed))**.5)

linear regression : 4011095.5641239663


#### 중간 정리

gradient boost와 xgboost가 가장 좋은 성능을 보였고 light GBM 또한 비교적 적은 에러값을 보였다

모델 스태킹을 진행한다면 이들을 중심으로 하는 것이 좋을것으로 보인다

### Model Stacking

#### Build & Learn Stacking Transformer

In [34]:
estimators = [  # RMSE < 0.9M
    ('gbr_g', ensemble.GradientBoostingRegressor(**gbr_g_param))
    , ('gbr_r', ensemble.GradientBoostingRegressor(**gbr_r_param))
    # , ('gbr_b', ensemble.GradientBoostingRegressor(**gbr_b_param))
    , ('xgbr_g', XGBRegressor(**xgbr_g_param))
    , ('xgbr_r', XGBRegressor(**xgbr_r_param))
    , ('xgbr_b', XGBRegressor(**xgbr_b_param))
    # , ('xtr_b', ensemble.ExtraTreesRegressor(**xtr_b_param))
    # , ('rfr_b', ensemble.RandomForestRegressor(**rfr_b_param))
    # , ('lgbr_r', lightgbm.LGBMRegressor(**lgbr_r_param))
    # , ('lgbr_b', lightgbm.LGBMRegressor(**lgbr_b_param))
]

In [35]:
stack = StackingTransformer(estimators, 
                            regression = True, 
                            metric = mean_squared_error, 
                            n_folds = 10, shuffle = True, 
                            random_state = 0, verbose = 1)

In [36]:
stack = stack.fit(x_train_transformed, y_train)

task:         [regression]
metric:       [mean_squared_error]
variant:      [A]
n_estimators: [5]

estimator  0: [gbr_g: GradientBoostingRegressor]
    MEAN:     [313710080744.95159912] + [214377429522.06961060]

estimator  1: [gbr_r: GradientBoostingRegressor]
    MEAN:     [327538638083.39758301] + [208350963033.26617432]

estimator  2: [xgbr_g: XGBRegressor]
[07:55:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:55:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:55:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:55:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:55:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[

#### Stacked Feature

In [37]:
s_train = stack.transform(x_train_transformed)
s_test = stack.transform(x_test_transformed)

Train set was detected.
Transforming...

estimator  0: [gbr_g: GradientBoostingRegressor]
    DONE

estimator  1: [gbr_r: GradientBoostingRegressor]
    DONE

estimator  2: [xgbr_g: XGBRegressor]
    DONE

estimator  3: [xgbr_r: XGBRegressor]
    DONE

estimator  4: [xgbr_b: XGBRegressor]
    DONE

Transforming...

estimator  0: [gbr_g: GradientBoostingRegressor]
    DONE

estimator  1: [gbr_r: GradientBoostingRegressor]
    DONE

estimator  2: [xgbr_g: XGBRegressor]
    DONE

estimator  3: [xgbr_r: XGBRegressor]
    DONE

estimator  4: [xgbr_b: XGBRegressor]
    DONE



In [40]:
xgbr_param_grid = {
    'n_estimators' : Integer(10, 1000, 'log-uniform')
    , 'gamma' : Real(1e-3, 1e-1, 'log-uniform')
    , 'max_depth' : Integer(1, 8)
    , 'learning_rate' : Real(1e-3, 1e-1, 'log-uniform')
    , 'random_state' : [42]
}

xgbr_hpo = BayesSearchCV(XGBRegressor(), xgbr_param_grid, refit=True, verbose=2, n_jobs=-1, n_iter=72, cv=5)

xgbr_hpo.fit(s_train, y_train)
print('The best parameters are ', xgbr_hpo.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

In [41]:
second_model = XGBRegressor(**xgbr_hpo.best_params_).fit(s_train, y_train)

print('Error of Model Stacking', mean_squared_error(second_model.predict(s_test), y_test) ** .5)

[08:23:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Error of Model Stacking 1081899.1089048602


### 결과 수치 pkl 출력

In [45]:
res_dict = {
    'linear regression': mean_squared_error(y_test, linear.predict(x_test_transformed))**.5
    , 'svm / grid': mean_squared_error(y_test, svr_g.predict(x_test_transformed))**.5
    , 'svm / rand': mean_squared_error(y_test, svr_r.predict(x_test_transformed))**.5
    , 'svm / baye': mean_squared_error(y_test, svr_b.predict(x_test_transformed))**.5
    , 'linear svm / grid': mean_squared_error(y_test, lsvr_g.predict(x_test_transformed))**.5
    , 'linear svm / rand': mean_squared_error(y_test, lsvr_r.predict(x_test_transformed))**.5
    , 'linear svm / baye': mean_squared_error(y_test, lsvr_b.predict(x_test_transformed))**.5
    , 'extra trees / grid': mean_squared_error(y_test, xtr_g.predict(x_test_transformed))**.5
    , 'extra trees / rand': mean_squared_error(y_test, xtr_r.predict(x_test_transformed))**.5
    , 'extra trees / baye': mean_squared_error(y_test, xtr_b.predict(x_test_transformed))**.5
    , 'random forest / grid': mean_squared_error(y_test, rfr_g.predict(x_test_transformed))**.5
    , 'random forest / rand': mean_squared_error(y_test, rfr_r.predict(x_test_transformed))**.5
    , 'random forest / baye': mean_squared_error(y_test, rfr_b.predict(x_test_transformed))**.5
    , 'gradient boost / grid': mean_squared_error(y_test, gbr_g.predict(x_test_transformed))**.5
    , 'gradient boost / rand': mean_squared_error(y_test, gbr_r.predict(x_test_transformed))**.5
    , 'gradient boost / baye': mean_squared_error(y_test, gbr_b.predict(x_test_transformed))**.5
    , 'xgboost / grid': mean_squared_error(y_test, xgbr_g.predict(x_test_transformed))**.5
    , 'xgboost / rand': mean_squared_error(y_test, xgbr_r.predict(x_test_transformed))**.5
    , 'xgboost / baye': mean_squared_error(y_test, xgbr_b.predict(x_test_transformed))**.5
    , 'light gbm / grid': mean_squared_error(y_test, lgbr_g.predict(x_test_transformed))**.5
    , 'light gbm / rand': mean_squared_error(y_test, lgbr_r.predict(x_test_transformed))**.5
    , 'light gbm / baye': mean_squared_error(y_test, lgbr_b.predict(x_test_transformed))**.5
    , 'sgd regression / grid': mean_squared_error(y_test, sgdr_g.predict(x_test_transformed))**.5
    , 'sgd regression / rand': mean_squared_error(y_test, sgdr_r.predict(x_test_transformed))**.5
    , 'sgd regression / baye': mean_squared_error(y_test, sgdr_b.predict(x_test_transformed))**.5
    , 'model stacking / baye': mean_squared_error(second_model.predict(s_test), y_test) ** .5
}

joblib.dump(res_dict, 'res_ml.pkl')

['res_ml.pkl']

In [46]:
res_pycrt = {
    '1. stat_potential - stat_overall' : 446
    , '2. (stat_potential - stat_overall) * stat_potential' : 431
    , '3. (stat_potential - stat_overall) * stat_overall' : 426
    , '4. (stat_potential - stat_overall) * age' : 434
    , '5. stat_potential^2 - stat_overall^2' : 427
    , '6. stat_potential^2 / stat_overall^2' : 440
    , '7. stat_potential * stat_overall * age' : 421
    , '8. stat_potential^2 / stat_overall / age' : 438
    , '9. stat_potential^2 / stat_overall^2' : 439
    , '10. stat_potential / stat_overall' : 440
    , '11. (stat_potential^2 - stat_overall^2) * age' : 435
}

joblib.dump(res_pycrt, 'res_pycrt.pkl')

['res_pycrt.pkl']